In [1]:
import geopandas as gpd
import momepy as mm
from tqdm import tqdm
from momepy import limit_range
import numpy as np
import pandas as pd
from inequality.theil import Theil
import libpysal
import scipy as sp
import mapclassify
import mapclassify.classifiers as classifiers

In [2]:
blg = gpd.read_parquet('../../nairobi/buildings.pq')
streets = gpd.read_parquet('../../nairobi/edges.pq')
tess = gpd.read_parquet('../../nairobi/tessellation.pq')
blocks = gpd.read_parquet('../../nairobi/blocks.pq')

## Measure

In [3]:
blg['sdbAre'] = mm.Area(blg).series
blg['sdbPer'] = mm.Perimeter(blg).series
blg['ssbCCo'] = mm.CircularCompactness(blg, 'sdbAre').series
blg['ssbCor'] = mm.Corners(blg).series
blg['ssbSqu'] = mm.Squareness(blg).series
blg['ssbERI'] = mm.EquivalentRectangularIndex(blg, 'sdbAre', 'sdbPer').series
blg['ssbElo'] = mm.Elongation(blg).series

 46%|████▌     | 230978/507532 [00:32<00:39, 7017.49it/s]/opt/conda/lib/python3.8/site-packages/momepy/shape.py:773: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
 46%|████▌     | 231102/507532 [00:52<01:01, 4523.21it/s]/opt/conda/lib/python3.8/site-packages/momepy/shape.py:862: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))
100%|██████████| 507532/507532 [01:59<00:00, 4237.94it/s]


In [4]:
cencon = mm.CentroidCorners(blg)
blg['ssbCCM'] = cencon.mean
blg['ssbCCD'] = cencon.std

 46%|████▌     | 231144/507532 [01:07<01:22, 3366.12it/s]/opt/conda/lib/python3.8/site-packages/momepy/shape.py:1080: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
100%|██████████| 507532/507532 [02:32<00:00, 3325.20it/s]


In [5]:
blg['stbOri'] = mm.Orientation(blg).series
tess['stcOri'] = mm.Orientation(tess).series
blg['stbCeA'] = mm.CellAlignment(blg, tess, 'stbOri', 'stcOri', 'uID', 'uID').series

100%|██████████| 506435/506435 [06:32<00:00, 1290.98it/s]


In [6]:
tess['sdcLAL'] = mm.LongestAxisLength(tess).series
tess['sdcAre'] = mm.Area(tess).series
tess['sscCCo'] = mm.CircularCompactness(tess, 'sdcAre').series
tess['sscERI'] = mm.EquivalentRectangularIndex(tess, 'sdcAre').series
tess['sicCAR'] = mm.AreaRatio(tess, blg, 'sdcAre', 'sdbAre', 'uID').series

In [7]:
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tess, ids="uID", silence_warnings=True)
 
blg["mtbAli"] = mm.Alignment(blg, queen_1, "uID", "stbOri").series
blg["mtbNDi"] = mm.NeighborDistance(blg, queen_1, "uID").series
tess["mtcWNe"] = mm.Neighbors(tess, queen_1, "uID", weighted=True).series
tess["mdcAre"] = mm.CoveredArea(tess, queen_1, "uID").series

100%|██████████| 506435/506435 [01:25<00:00, 5941.44it/s]


In [9]:
blocks["ldkAre"] = mm.Area(blocks).series
blocks["ldkPer"] = mm.Perimeter(blocks).series
blocks["lskCCo"] = mm.CircularCompactness(blocks, "ldkAre").series
blocks["lskERI"] = mm.EquivalentRectangularIndex(blocks, "ldkAre", "ldkPer").series
blocks["lskCWA"] = mm.CompactnessWeightedAxis(blocks, "ldkAre", "ldkPer").series
blocks["ltkOri"] = mm.Orientation(blocks).series
 
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID", silence_warnings=True)
 
blocks["ltkWNB"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True).series
blocks["likWBB"] = mm.Count(blocks, blg, "bID", "bID", weighted=True).series

100%|██████████| 12191/12191 [00:00<00:00, 96794.87it/s]


In [11]:
tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('../../nairobi/blg_data.parquet')
blocks.drop(columns='geometry').to_parquet('../../nairobi/blocks_data.parquet')

<ipython-input-11-5b763fe89c32>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
<ipython-input-11-5b763fe89c32>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend us

In [12]:
queen3 = mm.sw_high(k=3, weights=queen_1)
queen1 = queen_1
blg_queen = blg_q1

blg['ltbIBD'] = mm.MeanInterbuildingDistance(blg, queen1, 'uID', queen3).series
blg['ltcBuA'] = mm.BuildingAdjacency(blg, queen3, 'uID', blg_queen).series

  0%|          | 324/507532 [00:00<05:12, 1625.52it/s]

Computing mean interbuilding distances...


Calculating adjacency: 100%|██████████| 507532/507532 [00:03<00:00, 156483.19it/s]


In [13]:
tess = tess.merge(blg[['uID']], on='uID', how='left')

tess['ltcWRB'] = mm.BlocksCount(tess, 'bID', queen3, 'uID').series

100%|██████████| 506435/506435 [06:22<00:00, 1325.50it/s]


In [14]:
tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('../../nairobi/blg_data.parquet')
 
fo = libpysal.io.open('../../nairobi/queen1.gal', 'w')
fo.write(queen1)
fo.close()
 
fo = libpysal.io.open('../../nairobi/queen3.gal', 'w')
fo.write(queen3)
fo.close()
 
fo = libpysal.io.open('../../nairobi/blg_queen.gal', 'w')
fo.write(blg_queen)
fo.close()

<ipython-input-14-bd7e78ee6eca>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
<ipython-input-14-bd7e78ee6eca>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend us

In [15]:
streets["sdsLen"] = mm.Perimeter(streets).series
tess["stcSAl"] = mm.StreetAlignment(tess, streets, "stcOri", "nID").series
blg["stbSAl"] = mm.StreetAlignment(blg, streets, "stbOri", "nID").series

profile = mm.StreetProfile(streets, blg, distance=3)
streets["sdsSPW"] = profile.w
streets["sdsSPO"] = profile.o
streets["sdsSWD"] = profile.wd
 
streets["sssLin"] = mm.Linearity(streets).series
streets["sdsAre"] = mm.Reached(streets, tess, "nID", "nID", mode="sum", values="sdcAre").series
streets["sisBpM"] = mm.Count(streets, blg, "nID", "nID", weighted=True).series

/opt/conda/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/lib/python3.8/site-packages/momepy/dimension.py:626: RuntimeWarning: invalid value encountered in long_scalars
  openness.append(np.isnan(s).sum() / (f).sum())
100%|██████████| 115518/115518 [05:14<00:00, 366.90it/s]


In [16]:
tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('../../nairobi/blg_data.parquet')
streets.drop(columns='geometry').to_parquet('../../nairobi/streets_data.parquet')

<ipython-input-16-6c060e936c79>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
<ipython-input-16-6c060e936c79>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend us

In [17]:
str_q1 = libpysal.weights.contiguity.Queen.from_dataframe(streets)
 
streets["misRea"] = mm.Reached(
    streets, tess, "nID", "nID", spatial_weights=str_q1, mode="count"
).series
streets["mdsAre"] = mm.Reached(streets, tess, "nID", "nID", spatial_weights=str_q1,
                               mode="sum").series

100%|██████████| 115518/115518 [14:26<00:00, 133.26it/s]


In [18]:
graph = mm.gdf_to_nx(streets)
 
print("node degree")
graph = mm.node_degree(graph)
 
print("subgraph")
graph = mm.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
)
print("cds length")
graph = mm.cds_length(graph, radius=3, name="ldsCDL")
 
print("clustering")
graph = mm.clustering(graph, name="xcnSCl")
 
print("mean_node_dist")
graph = mm.mean_node_dist(graph, name="mtdMDi")
 
nodes, edges, sw = mm.nx_to_gdf(graph, spatial_weights=True)
 
print("saving")
nodes.to_parquet('../../nairobi/g_nodes.pq')
edges.to_parquet('../../nairobi/g_edges.pq')
 
fo = libpysal.io.open('../../nairobi/nodes.gal', "w")
fo.write(sw)
fo.close()
 
edges_w3 = mm.sw_high(k=3, gdf=edges)
edges["ldsMSL"] = mm.SegmentsLength(edges, spatial_weights=edges_w3, mean=True).series
 
edges["ldsRea"] = mm.Reached(edges, tess, "nID", "nID", spatial_weights=edges_w3).series
edges["ldsRea"] = mm.Reached(
    edges, tess, "nID", "nID", spatial_weights=edges_w3, mode="sum", values="sdcAre"
).series
 
nodes_w5 = mm.sw_high(k=5, weights=sw)
nodes["lddNDe"] = mm.NodeDensity(nodes, edges, nodes_w5).series
nodes["linWID"] = mm.NodeDensity(
    nodes, edges, nodes_w5, weighted=True, node_degree="degree"
).series
 
blg["nodeID"] = mm.get_node_id(blg, nodes, edges, "nodeID", "nID")
tess = tess.merge(blg[["uID", "nodeID"]], on="uID", how="left")
 
nodes_w3 = mm.sw_high(k=3, weights=sw)
 
nodes["lddRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", nodes_w3).series
nodes["lddARe"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", nodes_w3, mode="sum", values="sdcAre"
).series
 
nodes["sddAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", mode="sum", values="sdcAre"
).series
nodes["midRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", spatial_weights=sw).series
nodes["midAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", spatial_weights=sw, mode="sum", values="sdcAre"
).series
 
nodes.rename(
    columns={
        "degree": "mtdDeg",
        "meshedness": "lcdMes",
        "local_closeness": "lcnClo",
        "proportion_3": "linP3W",
        "proportion_4": "linP4W",
        "proportion_0": "linPDE",
    }, inplace=True
)

node degree
subgraph


100%|██████████| 53367/53367 [01:12<00:00, 739.29it/s] 


cds length


100%|██████████| 53367/53367 [00:27<00:00, 1938.84it/s]


clustering
mean_node_dist


100%|██████████| 53367/53367 [00:00<00:00, 68858.41it/s]


saving


<ipython-input-18-5e698dbfc976>:35: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  nodes.to_parquet('../../nairobi/g_nodes.pq')
<ipython-input-18-5e698dbfc976>:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production sett

In [19]:
tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('../../nairobi/blg_data.parquet')
nodes.drop(columns='geometry').to_parquet('../../nairobi/nodes_data.parquet')
edges.drop(columns='geometry').to_parquet('../../nairobi/edges_data.parquet')

<ipython-input-19-f1c2eb66ab77>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  tess.drop(columns='geometry').to_parquet('../../nairobi/tess_data.parquet')
<ipython-input-19-f1c2eb66ab77>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend us

In [20]:
merged = tess.merge(blg.drop(columns=['nID', 'bID', 'nodeID', 'geometry']), on='uID')
merged = merged.merge(blocks.drop(columns='geometry'), on='bID', how='left')
merged = merged.merge(edges.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')

In [21]:
primary = merged.drop(columns=['nID', 'bID', 'nodeID', 'mm_len', 'cdsbool', 'node_start', 'node_end', 'geometry'])
primary.to_parquet('../../nairobi/primary.parquet')

<ipython-input-21-52050a5a525f>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  primary.to_parquet('../../nairobi/primary.parquet')


## Contextual

In [22]:
def theil(y):
    y = np.array(y)
    n = len(y)
    plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
    yt = plus.sum(axis=0)
    s = plus / (yt * 1.0)
    lns = np.log(n * s)
    slns = s * lns
    t = sum(slns)
    return t

In [2]:
def _simpson_di(data):

    def p(n, N):
        if n == 0:
            return 0
        return float(n) / N

    N = sum(data.values())

    return sum(p(n, N) ** 2 for n in data.values() if n != 0)

In [3]:
primary = pd.read_parquet('../../nairobi/primary.parquet')

In [4]:
queen3 = libpysal.io.open('../../nairobi/queen3.gal').read()
queen3.neighbors = {int(k): [int(i) for i in v] for k, v in queen3.neighbors.items()}  # we need values as integers again

/opt/conda/lib/python3.8/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 139 disconnected components.
 There are 71 islands with ids: 3015, 11206, 13697, 13843, 14610, 15499, 18012, 19160, 19929, 20753, 20848, 24983, 29611, 32447, 36310, 119946, 165193, 168594, 174847, 175116, 177539, 187644, 188706, 190477, 191837, 335522, 345106, 351469, 365303, 408100, 410691, 411858, 412434, 421880, 431056, 432690, 435297, 440087, 442249, 444120, 444560, 446730, 452966, 455208, 458582, 464477, 476583, 484954, 485738, 489098, 496837, 500059, 501163, 501321, 501441, 501614, 502050, 502394, 503034, 503630, 503905, 503950, 505797, 506779, 507048, 507143, 507325, 507472, 507474, 507510, 507511.
  warnings.warn(message)


In [5]:
gdf = primary
spatial_weights = queen3

In [6]:
gdf = gdf.set_index('uID')
unique_id = 'uID'

In [41]:
means = {}
ranges = {}
theils = {}

In [42]:
for ch in gdf.columns:
    means[ch] = []
    ranges[ch] = []
    theils[ch] = []

In [12]:
gdf = gdf.fillna(0)  # normally does not happen, but to be sure
chars = gdf.columns

In [10]:
gdf['lcdMes'] = gdf.apply(
            lambda row: row.lcdMes if row.lcdMes >= 0 else 0,
            axis=1,
        )  # normally does not happen, but to be sure

In [8]:
gdf = gdf.round(6)

In [9]:
gdf = gdf.drop(columns="highway")

In [ ]:
for index, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
    neighbours = [index]
    neighbours += spatial_weights.neighbors[index]
    
    for ch in gdf.columns:
        values_list = gdf.loc[neighbours][ch] 
        idec = limit_range(values_list, rng=(10, 90))
        iquar = limit_range(values_list, rng=(25, 75))
        
        means[ch].append(np.mean(iquar))
        ranges[ch].append(sp.stats.iqr(values_list, rng=(25, 75)))
        theils[ch].append(theil(idec))

 26%|██▋       | 133685/506435 [1:50:46<5:08:56, 20.11it/s]

In [ ]:
for ch in gdf.columns:
    gdf[ch + '_meanIQ3'] = means[ch]
    gdf[ch + '_rangeIQ3'] = ranges[ch]
    gdf[ch + '_theilID3'] = theils[ch]

In [13]:
skewness = pd.DataFrame(index=chars)
for c in chars:
    skewness.loc[c, 'skewness'] = sp.stats.skew(gdf[c])
headtail = list(skewness.loc[skewness.skewness >= 1].index)
to_invert = skewness.loc[skewness.skewness <= -1].index

for inv in to_invert:
    gdf[inv + '_r'] = gdf[inv].max() - gdf[inv]
inverted = [x for x in gdf.columns if '_r' in x]
headtail = headtail + inverted
natural = [x for x in chars if x not in headtail]

In [14]:
schemes = {}
for classifier in classifiers.CLASSIFIERS:
    schemes[classifier.lower()] = getattr(classifiers, classifier)

In [15]:
results = {}
for c in headtail + natural:
    results[c] = []
bins = {}
for c in headtail:
    bins[c] = schemes['headtailbreaks'](gdf[c]).bins
for c in natural:
    bins[c] = mapclassify.gadf(gdf[c], method='NaturalBreaks')[1].bins

/opt/conda/lib/python3.8/site-packages/mapclassify/classifiers.py:1712: UserWarning: Warning: Not enough unique values in array to form k classes
  Warn(ms, UserWarning)
/opt/conda/lib/python3.8/site-packages/mapclassify/classifiers.py:1713: UserWarning: Warning: setting k to 1
  Warn("Warning: setting k to %d" % uvk, UserWarning)
/opt/conda/lib/python3.8/site-packages/mapclassify/classifiers.py:888: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
/opt/conda/lib/python3.8/site-packages/mapclassify/classifiers.py:2644: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - cl.adcm / adam


In [16]:
for index, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
    neighbours = [index]
    neighbours += spatial_weights.neighbors[index]
    
    subset = gdf.loc[neighbours]
    for c in headtail + natural:
        values = subset[c]
        sample_bins = classifiers.UserDefined(values, list(bins[c]))
        counts = dict(zip(bins[c], sample_bins.counts))
        results[c].append(_simpson_di(counts))

  0%|          | 0/506435 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/mapclassify/classifiers.py:888: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
100%|██████████| 506435/506435 [3:39:42<00:00, 38.42it/s]  


In [17]:
for c in headtail + natural:
    gdf[c + '_simpson'] = results[c]

In [18]:
gdf.columns

Index(['stcOri', 'sdcLAL', 'sdcAre', 'sscCCo', 'sscERI', 'sicCAR', 'mtcWNe',
       'mdcAre', 'ltcWRB', 'stcSAl',
       ...
       'lskERI_simpson', 'ltkOri_simpson', 'sdsSPW_simpson', 'sdsSPO_simpson',
       'sdsSWD_simpson', 'sssLin_simpson', 'mtdDeg_simpson', 'lcdMes_simpson',
       'linPDE_simpson', 'lcnClo_simpson'],
      dtype='object', length=130)

In [ ]:
gdf.rename(columns={'sscERI_r_simpson': 'sscERI_simpson', 'ssbERI_r_simpson': 'ssbERI_simpson',}, inplace=True)

In [19]:
pat = [x for x in gdf.columns if '_' in x]
gdf2 = gdf[[x for x in pat if 'uID' not in x]].reset_index()

In [ ]:
gdf2.columns.duplicated()

In [20]:
gdf2.to_parquet('../../nairobi/contextual_simpson.parquet')

In [49]:
gdf2

,uID,stcOri_meanIQ3,stcOri_rangeIQ3,stcOri_theilID3,sdcLAL_meanIQ3,sdcLAL_rangeIQ3,sdcLAL_theilID3,sdcAre_meanIQ3,sdcAre_rangeIQ3,sdcAre_theilID3,...,lddARe_theilID3,sddAre_meanIQ3,sddAre_rangeIQ3,sddAre_theilID3,midRea_meanIQ3,midRea_rangeIQ3,midRea_theilID3,midAre_meanIQ3,midAre_rangeIQ3,midAre_theilID3
0,0,8.690215,13.745575,0.333951,115.638400,43.197802,0.033617,5260.655169,7588.760250,0.174004,...,0.008113,84128.089976,54502.216490,0.082711,56.000000,41.00,0.055547,209051.361345,81741.748888,0.031183
1,1,17.107346,28.356627,0.364632,60.961918,70.039744,0.104780,1255.973917,2621.825395,0.303457,...,0.008941,16348.304121,6578.714927,0.015081,46.291667,39.00,0.068966,44383.228233,22021.813914,0.036167
2,2,7.699236,16.851171,0.407314,45.145050,32.446339,0.099529,717.559132,932.747991,0.469423,...,0.807105,16615.312454,3376.471169,0.042167,32.838710,11.00,0.071209,32471.304686,24619.212867,0.086180
3,3,19.012996,19.053156,0.119368,95.261736,47.425937,0.032151,2701.697845,2189.293257,0.106185,...,0.003860,44432.908576,11912.247415,0.028789,49.000000,18.00,0.030669,132284.092048,54797.623870,0.026767
4,4,22.519366,22.271200,0.193274,78.545408,65.914871,0.081129,1827.189199,3459.816373,0.332190,...,0.030171,46195.334468,7346.767908,0.003133,84.478261,100.00,0.173431,134231.364619,98437.630441,0.065439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506430,507527,19.687929,15.156462,0.128418,184.452129,47.343257,0.008062,14048.544773,11533.782290,0.111906,...,0.010786,397608.558320,62749.083652,0.003111,59.923077,11.00,0.004213,874887.395962,156923.244976,0.004022
506431,507528,12.812096,18.004769,0.198687,106.529893,69.976596,0.060260,3816.254174,5079.873545,0.214102,...,0.022618,28891.725607,10655.582934,0.026913,19.782609,19.25,0.176708,45391.649619,49578.479856,0.100455
506432,507529,33.110922,24.847249,0.113949,54.269134,60.121538,0.132972,1034.006488,2241.823786,0.398487,...,0.088036,21272.508388,40706.759142,0.338119,22.795455,19.00,0.421368,61686.671468,56209.577352,0.107780
506433,507530,16.393865,27.536475,0.298553,105.057072,76.002043,0.090215,3286.633690,4229.604974,0.239890,...,0.018372,67684.048652,27161.168313,0.214722,19.250000,28.00,0.554787,86233.695861,0.000000,0.203550
